# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV

In [3]:
file_path = "Resources/online_shoppers_intention.csv"
shoppers_df = pd.read_csv(file_path)
shoppers_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [4]:
# Changed Revenue to dtype str
shoppers_df['Revenue'] = shoppers_df['Revenue'].astype('str')

# Replace True and False in Column Revenue by Sale or No Sale for better readability
shoppers_df['Revenue'] = shoppers_df['Revenue'].replace(['True'],'Sale')
shoppers_df['Revenue'] = shoppers_df['Revenue'].replace(['False'],'No Sale')

In [5]:
shoppers_df

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,Feb,1,1,1,1,Returning_Visitor,False,No Sale
1,0,0.0,0,0.0,2,64.000000,0.000000,0.100000,0.000000,0.0,Feb,2,2,1,2,Returning_Visitor,False,No Sale
2,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,Feb,4,1,9,3,Returning_Visitor,False,No Sale
3,0,0.0,0,0.0,2,2.666667,0.050000,0.140000,0.000000,0.0,Feb,3,2,2,4,Returning_Visitor,False,No Sale
4,0,0.0,0,0.0,10,627.500000,0.020000,0.050000,0.000000,0.0,Feb,3,3,1,4,Returning_Visitor,True,No Sale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,3,145.0,0,0.0,53,1783.791667,0.007143,0.029031,12.241717,0.0,Dec,4,6,1,1,Returning_Visitor,True,No Sale
12326,0,0.0,0,0.0,5,465.750000,0.000000,0.021333,0.000000,0.0,Nov,3,2,1,8,Returning_Visitor,True,No Sale
12327,0,0.0,0,0.0,6,184.250000,0.083333,0.086667,0.000000,0.0,Nov,3,2,1,13,Returning_Visitor,True,No Sale
12328,4,75.0,0,0.0,15,346.000000,0.000000,0.021053,0.000000,0.0,Nov,2,2,3,11,Returning_Visitor,False,No Sale


# Split the Data into Training and Testing

In [6]:
# Create our features
X = shoppers_df.drop("Revenue", axis=1)
X = pd.get_dummies(X)


# Create our target
target = ["Revenue"]

y = shoppers_df.loc[:, target].copy()

In [7]:
X.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Jul,Month_June,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,VisitorType_New_Visitor,VisitorType_Other,VisitorType_Returning_Visitor
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,...,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,...,0.035036,0.023358,0.154663,0.272830,0.243147,0.044526,0.036334,0.137388,0.006894,0.855718
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,...,0.183880,0.151043,0.361598,0.445432,0.429000,0.206268,0.187128,0.344271,0.082745,0.351390
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# Check the balance of our target values
y['Revenue'].value_counts()

No Sale    10422
Sale        1908
Name: Revenue, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print("Training:",Counter(y_train['Revenue']))
print("Test:",Counter(y_test['Revenue']))

Training: Counter({'No Sale': 7786, 'Sale': 1461})
Test: Counter({'No Sale': 2636, 'Sale': 447})


# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [10]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
print(Counter(y_resampled['Revenue']))

Counter({'No Sale': 7786, 'Sale': 7786})


In [11]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [12]:
# Calculate predictions
y_pred = model.predict(X_test)

# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test,y_pred)

0.8195230893530636

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cmatrix = confusion_matrix(y_test, y_pred)

# This will show the confusion matrix as a Pandas Dataframe for better readability
cmatrix_df = pd.DataFrame(
    cmatrix, index=["Actual No Sale", "Actual Sale"], columns=["Predicted No Sale", "Predicted Sale"])

cmatrix_df

,Predicted No Sale,Predicted Sale
Actual No Sale,2345,291
Actual Sale,112,335


In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

# This will show the classification report as a Pandas Dataframe for better readability
# Source: https://stackoverflow.com/questions/39662398/scikit-learn-output-metrics-classification-report-into-csv-tab-delimited-format'''
from sklearn import metrics
report = metrics.classification_report(y_test, y_pred, output_dict=True)
df_classification_report = pd.DataFrame(report).transpose()
df_classification_report = df_classification_report.sort_values(by=['f1-score'], ascending=False)

df_classification_report

                   pre       rec       spe        f1       geo       iba       sup

    No Sale       0.95      0.89      0.75      0.92      0.82      0.68      2636
       Sale       0.54      0.75      0.89      0.62      0.82      0.66       447

avg / total       0.89      0.87      0.77      0.88      0.82      0.67      3083



,precision,recall,f1-score,support
No Sale,0.954416,0.889605,0.920872,2636.000000
weighted avg,0.893626,0.869283,0.877889,3083.000000
accuracy,0.869283,0.869283,0.869283,0.869283
macro avg,0.744780,0.819523,0.772645,3083.000000
Sale,0.535144,0.749441,0.624418,447.000000


### SMOTE Oversampling

In [15]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
print(Counter(y_resampled['Revenue']))

Counter({'No Sale': 7786, 'Sale': 7786})


In [16]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [17]:
# Calculate predictions
y_pred = model.predict(X_test)

In [18]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.8086832465976176

In [19]:
# Display the confusion matrix
cmatrix2 = confusion_matrix(y_test, y_pred)

cmatrix2

array([[2394,  242],
       [ 130,  317]], dtype=int64)

In [20]:
# This will show the confussion matrix as a Pandas Dataframe for improved readability

cmatrix_df2 = pd.DataFrame(cmatrix2, index=["Actual No Sale", "Actual Sale"],
                                   columns=["Predicted No Sale", "Predicted Sale"])

cmatrix_df2

,Predicted No Sale,Predicted Sale
Actual No Sale,2394,242
Actual Sale,130,317


In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

    No Sale       0.95      0.91      0.71      0.93      0.80      0.66      2636
       Sale       0.57      0.71      0.91      0.63      0.80      0.63       447

avg / total       0.89      0.88      0.74      0.88      0.80      0.65      3083



In [22]:
# This will show the classification report as a Pandas Dataframe for better readability
# Source: https://stackoverflow.com/questions/39662398/scikit-learn-output-metrics-classification-report-into-csv-tab-delimited-format'''
from sklearn import metrics
report2 = metrics.classification_report(y_test, y_pred, output_dict=True)
df_classification_report2 = pd.DataFrame(report2).transpose()
df_classification_report2 = df_classification_report2.sort_values(by=['f1-score'], ascending=False)

df_classification_report2


,precision,recall,f1-score,support
No Sale,0.948494,0.908194,0.927907,2636.000000
weighted avg,0.893194,0.879338,0.884746,3083.000000
accuracy,0.879338,0.879338,0.879338,0.879338
macro avg,0.757789,0.808683,0.779063,3083.000000
Sale,0.567084,0.709172,0.630219,447.000000


# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [23]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cluster_centroids = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cluster_centroids.fit_resample(X_train, y_train)
print(Counter(y_resampled["Revenue"]))

Counter({'No Sale': 1461, 'Sale': 1461})


In [24]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [25]:
# Calculate predictions
y_pred = model.predict(X_test)

In [26]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.5410848923696333

In [27]:
# Display the confusion matrix
cmatrix3 = confusion_matrix(y_test, y_pred)

cmatrix3

array([[ 824, 1812],
       [ 103,  344]], dtype=int64)

In [28]:
# This will show the classification report as a Pandas Dataframe for better readability
cmatrix3_df = pd.DataFrame(
    cmatrix3, index=["Actual No Sale", "Actual Sale"], columns=["Predicted No Sale", "Predicted Sale"])

cmatrix3_df

,Predicted No Sale,Predicted Sale
Actual No Sale,824,1812
Actual Sale,103,344


In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

    No Sale       0.89      0.31      0.77      0.46      0.49      0.23      2636
       Sale       0.16      0.77      0.31      0.26      0.49      0.25       447

avg / total       0.78      0.38      0.70      0.43      0.49      0.23      3083



In [30]:
# This will show the classification report as a Pandas Dataframe for better readability
# Source: https://stackoverflow.com/questions/39662398/scikit-learn-output-metrics-classification-report-into-csv-tab-delimited-format'''
from sklearn import metrics
report3 = metrics.classification_report(y_test, y_pred, output_dict=True)
df_classification_report3 = pd.DataFrame(report3).transpose()
df_classification_report3 = df_classification_report3.sort_values(by=['f1-score'], ascending=False)

df_classification_report3


,precision,recall,f1-score,support
No Sale,0.888889,0.312595,0.462532,2636.000000
weighted avg,0.783144,0.378852,0.433792,3083.000000
accuracy,0.378852,0.378852,0.378852,0.378852
macro avg,0.524222,0.541085,0.363421,3083.000000
Sale,0.159555,0.769575,0.264310,447.000000


# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smoteenn_ = SMOTEENN(random_state=0)
X_resampled, y_resampled = smoteenn_.fit_resample(X, y)
print(Counter(y_resampled['Revenue']))

Counter({'Sale': 8908, 'No Sale': 7264})


In [32]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [33]:
# Calculate predictions
y_pred = model.predict(X_test)

In [34]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.8302848529906

In [35]:
# Display the confusion matrix
cmatrix4 = confusion_matrix(y_test, y_pred)

cmatrix4

array([[2272,  364],
       [  90,  357]], dtype=int64)

In [36]:
# This will show the classification report as a Pandas Dataframe for better readability
cmatrix4_df = pd.DataFrame(
    cmatrix4, index=["Actual No Sale", "Actual Sale"], columns=["Predicted No Sale", "Predicted Sale"])

cmatrix4_df

,Predicted No Sale,Predicted Sale
Actual No Sale,2272,364
Actual Sale,90,357


In [37]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

    No Sale       0.96      0.86      0.80      0.91      0.83      0.69      2636
       Sale       0.50      0.80      0.86      0.61      0.83      0.68       447

avg / total       0.89      0.85      0.81      0.87      0.83      0.69      3083



In [38]:
# This will show the classification report as a Pandas Dataframe for better readability
# Source: https://stackoverflow.com/questions/39662398/scikit-learn-output-metrics-classification-report-into-csv-tab-delimited-format'''
from sklearn import metrics
report4 = metrics.classification_report(y_test, y_pred, output_dict=True)
df_classification_report4 = pd.DataFrame(report4).transpose()
df_classification_report4 = df_classification_report4.sort_values(by=['f1-score'], ascending=False)

df_classification_report4

,precision,recall,f1-score,support
No Sale,0.961897,0.861912,0.909164,2636.000000
weighted avg,0.894223,0.852741,0.865977,3083.000000
accuracy,0.852741,0.852741,0.852741,0.852741
macro avg,0.728521,0.830285,0.760233,3083.000000
Sale,0.495146,0.798658,0.611301,447.000000
